
# A kutyák futási teljesítményére szignifikáns hatással van-e az, hogy ivaros/ivartalan kanra/szukára vált rá a kutya?
- a nem tisztitott, szélesformátumú táblából indulunk ki:flyballdf.csv
- a 3 váltás szerint szedjük szét a táblát és tesszük egymás alá a hasonló oszlopokat
- ugyanaz a kutya az egyik váltás induló kutyája (before_dog), míg a következő váltás befutó kutyája(after_dog)
- az ivarosság ill. nemre utaló jellemzőket mindegyikhez hozzáfűzzük
- a kapott hosszúformátumú táblát tisztítani is kell
- majd megfelelően csoportosítani (groupby) a futási értékek alapján

In [4]:
#Szélesformátumú adatbázis betöltése:
import pandas as pd
fly2 = pd.read_csv('./../data/flyballdf.csv', index_col = [0])


In [6]:
#kiválogatjuk csak a WildRunners csapatokat:
wildfly3 = fly2[fly2['who'].str.startswith('WildRunners')].copy()

#Ezekkel az oszlopokkal nem számolunk most:
wildfly3.drop(columns=['Division', 'with who', 'total time',
       'W/L/T', 'Hurdles', 'place', 'date', 'competition', 'when', 'who'], inplace=True)


In [4]:
#A 3 táblázat kijelölése, amit később egymás alá töltve egyszerre fogunk tisztitani, majd vizsgálni

wildflyA = wildfly3[['name', 'name.1', 'chng', '2.dog']]

wildflyB = wildfly3[['name.1', 'name.2', 'chng.1', '3.dog']]

wildflyC = wildfly3[['name.2', 'name.3', 'chng.2', '4.dog']]

In [5]:
#Betöltjük a kutyás adatbázist is, hogy az ivarosság és szuka/kan tulajdonságokat hozzá tudjuk adni

kutyak = pd.read_csv('./../data/kutyadf.csv', index_col=[0])

#Most csak a névre, a nem és az ivar oszlopokra van szükségünk
kutyakbefore = kutyak[['nev','nem','ivar']]
kutyakafter = kutyak[['nev','nem','ivar']]

#két ugyanolyan tábla, amiben más oszlopnevek lesznek, hogy megkülönböztessük a váltás előtt és a váltás után futó kutyát
kutyakbefore.columns = ['name_before','before_sex','before_gender']
kutyakafter.columns = ['name_after','after_sex','after_gender']

In [6]:
#Feltöltjük a kutyák neve mellett a neme és ivar oszlopokat, de két lépésben: először a before, utána az after kutyáét

wildflyA1 = pd.merge(wildflyA, kutyakbefore, left_on='name', right_on='name_before', how='inner')
wildflyA2 = pd.merge(wildflyA1, kutyakafter, left_on='name.1', right_on='name_after', how='inner')


wildflyB1 = pd.merge(wildflyB, kutyakbefore, left_on='name.1', right_on='name_before', how='inner')
wildflyB2 = pd.merge(wildflyB1, kutyakafter, left_on='name.2', right_on='name_after', how='inner')

wildflyC1 = pd.merge(wildflyC, kutyakbefore, left_on='name.2', right_on='name_before', how='inner')
wildflyC2 = pd.merge(wildflyC1, kutyakafter, left_on='name.3', right_on='name_after', how='inner')


In [7]:
#Összefűzzük az wildfly A2 - B2 - C2 táblákat, de előtte eldobjuk, majd egységesítjük az oszlopneveket

wildflyA2.drop(columns=['name', 'name.1'], inplace=True)
wildflyB2.drop(columns=['name.1', 'name.2'], inplace=True)
wildflyC2.drop(columns=['name.2', 'name.3'], inplace=True)

In [8]:
#Oszlopnevek egységesítése
wildflyA2.columns = ['relay','time','name_before','before_sex','before_gender','name_after','after_sex','after_gender']
wildflyB2.columns = ['relay','time','name_before','before_sex','before_gender','name_after','after_sex','after_gender']
wildflyC2.columns = ['relay','time','name_before','before_sex','before_gender','name_after','after_sex','after_gender']

In [9]:
#Egymás alá fűzzük a 3 táblát (ez a default: axis=0, új indexelést kap)
wildflyABC = pd.concat([wildflyA2, wildflyB2, wildflyC2], ignore_index=True)


In [10]:
#A váltás oszlopra nem vagyunk kiváncsiak:
wildflyABC.drop(columns=['relay'], inplace=True)

wildflyABC['time'].isna().sum() #11db helyen nincsen érték
wildflyABC['time'].unique() #mimo , vbehl, nan, outside

#ahol nincs érték eldobni:
wildflyABC.dropna(subset=["time"], inplace=True)

wildflyABC[wildflyABC['time'] == 'outside'] # 3181 sor
wildflyABC[wildflyABC['time'] == 'vběhl'] # 18 sor
wildflyABC[wildflyABC['time'] == 'mimo'] #32 sor

#a hibajelzéseket -999-re cserélni
wildflyABC['time'].replace('vběhl', -999, inplace=True)
wildflyABC['time'].replace('outside', -999, inplace=True)
wildflyABC['time'].replace('mimo', -999, inplace=True)

#adattípus csere, time oszlop legyen numerikus
wildflyABC['time'] = pd.to_numeric(wildflyABC['time'])

#eldobtuk mindent, ami a time oszlopban 3.3 alatt van, a -999re cserélt hibákkal együtt
wildflyABC = wildflyABC[wildflyABC['time'] > 3.3]

wildflyABC.head()

,time,name_before,before_sex,before_gender,name_after,after_sex,after_gender
0,5.20,Brix,kan,ivartalan,Them,kan,ivartalan
1,5.19,Brix,kan,ivartalan,Them,kan,ivartalan
2,5.48,Brix,kan,ivartalan,Them,kan,ivartalan
3,5.63,Brix,kan,ivartalan,Them,kan,ivartalan
4,5.51,Brix,kan,ivartalan,Them,kan,ivartalan


In [11]:
#Median futási idő szerint sorba rendezve kutyánként, függően attól, akire váltanak (nem, ivarosság)

sexgender = wildflyABC[['after_sex','before_sex', 'after_gender','before_gender',
                        'time']].groupby(['after_sex','after_gender',
                                          'before_sex', 'before_gender'] ).median().head(50).sort_values(by='time')
sexgender

time
after_sex after_gender before_sex before_gender       
szuka     ivaros       szuka      ivartalan      4.030
          ivartalan    szuka      ivaros         4.050
                                  ivartalan      4.110
kan       ivartalan    szuka      ivaros         4.200
szuka     ivaros       kan        ivartalan      4.240
kan       ivaros       szuka      ivartalan      4.280
                                  ivaros         4.290
szuka     ivartalan    kan        ivaros         4.290
          ivaros       kan        ivaros         4.320
          ivartalan    kan        ivartalan      4.380
kan       ivartalan    szuka      ivartalan      4.420
          ivaros       kan        ivaros         4.505
                                  ivartalan      4.510
          ivartalan    kan        ivartalan      4.660
                                  ivaros         4.740

In [13]:
#Legjobb futási idő szerint sorba rendezve
sexgendermin = wildflyABC[['after_sex','before_sex', 'after_gender','before_gender', 'time']].groupby(['after_sex','after_gender', 'before_sex', 'before_gender'] ).min().head(50).sort_values(by='time')
sexgendermin.to_excel('./../data/sexgendermin.xlsx')
sexgendermin

time
after_sex after_gender before_sex before_gender      
kan       ivartalan    kan        ivartalan      3.54
szuka     ivartalan    kan        ivartalan      3.79
                       szuka      ivartalan      3.79
kan       ivartalan    szuka      ivartalan      3.82
szuka     ivaros       szuka      ivartalan      3.82
          ivartalan    szuka      ivaros         3.82
                       kan        ivaros         3.88
kan       ivaros       szuka      ivaros         3.91
          ivartalan    kan        ivaros         3.91
          ivaros       kan        ivaros         3.96
          ivartalan    szuka      ivaros         3.99
          ivaros       szuka      ivartalan      4.00
                       kan        ivartalan      4.06
szuka     ivaros       kan        ivartalan      4.10
                                  ivaros         4.30

In [ ]:
#Szűrés arra, hogy a legjobb időknél, mi volt a 2 kutya neve, neme és az előző kutya ivarossága 
wildflyABC[['name_after', 'after_sex', 'after_gender','before_gender', 'name_before','time']].groupby(['name_after', 'name_before']).min().sort_values(by='time').head(10)
#Ez azért nem egyezik meg teljesen a legjobb idők listával, mert a groupolás során a nevek alá egybevonja ugyanazon párosításokat!

In [ ]:
#Ellenőrző python kód, hogy a groupolas során kapott minimum/medium futásidők jók-e
#Ez a kan-kan-ivartalan-ivartalan párosítások időeredményeit szedi listába
kankanlanlan = []
for ind in wildflyABC.index:
    if (wildflyABC['after_sex'][ind] == 'kan') & (wildflyABC['before_sex'][ind] == 'kan') & (wildflyABC['after_gender'][ind] == 'ivartalan') & (wildflyABC['before_gender'][ind] == 'ivartalan'):
        kankanlanlan.append(wildflyABC['time'][ind])
len(kankanlanlan)
import statistics
statistics.median(kankanlanlan)

In [ ]:
#Ez a szuka-szuka-ivaros-ivartalan futásidőket szedi ki listába
szuszuroslan = []
for ind in wildflyABC.index:
    if (wildflyABC['after_sex'][ind] == 'szuka') & (wildflyABC['before_sex'][ind] == 'szuka') & (wildflyABC['after_gender'][ind] == 'ivaros') & (wildflyABC['before_gender'][ind] == 'ivartalan'):
        szuszuroslan.append(wildflyABC['time'][ind])
len(szuszuroslan)
import statistics
statistics.median(szuszuroslan)

In [ ]:
#Melyik típusú váltásból mennyi volt összesen?
relaycount = wildflyABC[['after_sex','before_sex', 'after_gender','before_gender', 'time']].groupby(['after_sex','after_gender', 'before_sex', 'before_gender'] ).count().head(50).sort_values(by='time', ascending=False)
